In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
df_gscore = pd.read_csv('genome-scores.csv')
df_gtags = pd.read_csv('genome-tags.csv')
df_links = pd.read_csv('links.csv')
df_movies = pd.read_csv('movies.csv')
df_rating = pd.read_csv('ratings.csv')
df_tagspd = pd.read_csv('tags.csv')

## Creating the genome dataset

In [ ]:
df_gtagscore=df_gtags.merge(df_gscore, how='right', on='tagId').drop('tag', axis =1)
df_gtagscore.drop_duplicates(subset=['tagId', 'movieId'],inplace=True)
df_gtagscore.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Non-Null Count     Dtype  
---  ------     --------------     -----  
 0   tagId      11709768 non-null  int64  
 1   movieId    11709768 non-null  int64  
 2   relevance  11709768 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 357.4 MB


# pivoting the datafram

In [ ]:
df_gtagscore = df_gtagscore.pivot(index = 'movieId', columns = 'tagId', values='relevance')

In [ ]:
df_gtagscore.shape

(10381, 1128)

## adding realease year and genres as columns

In [ ]:
df_movies = pd.read_csv('movies.csv')
mask = df_movies['title'].str.contains('09–')

df_movies = pd.read_csv('movies.csv')
def condition(x):
    if x[-2:]=='a)':
        return np.nan
    elif x[-2:]=='l)':
        return np.nan
    elif x[-3:-1]=='7-':
        return 2007
    elif x[-4:-2]=='9–':
        return 2009
    elif x[-2:]=='))':
        return x[-6:-2]
    elif x[-1:]==')':
        return x[-5:-1]
    elif x[-1:]==' ':
        return x[-6:-2]
    else:
        return np.nan
df_movies['releaseyear'] = df_movies['title'].apply(condition).fillna(1993)

In [ ]:
df_movies.genres= df_movies.genres.str.split('|')
dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)
df_movies = pd.concat([df_movies, dummies], axis=1).drop('genres', axis=1)

C:\Users\HamidEsfahlani\AppData\Local\Temp\ipykernel_6668\1447042938.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
df_movies =df_movies.drop('title', axis =1)
scaler = MinMaxScaler()
df_movies.releaseyear = scaler.fit_transform(df_movies[['releaseyear']])

In [ ]:
df_ContBaseRec = pd.merge(df_gtagscore, df_movies, how='inner', on='movieId').set_index('movieId')
df_ContBaseRec.columns = df_ContBaseRec.columns.astype(str)

In [ ]:
df_ContBaseRec.head()

,1,2,3,4,5,6,7,8,9,10,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,...,0,0,0,0,0,0,0,0,0,0
2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,...,0,0,0,0,0,0,0,0,0,0
3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,...,0,0,0,0,0,1,0,0,0,0
4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,...,0,0,0,0,0,1,0,0,0,0
5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,...,0,0,0,0,0,0,0,0,0,0


# Clustering

## importing the libraries

In [ ]:
from sklearn.cluster import KMeans

## training the Model

In [ ]:
kmeans = KMeans(n_clusters=18, n_init='auto')
kmeans.fit(df_ContBaseRec)
labels = kmeans.labels_

In [ ]:
df_movies = pd.read_csv('movies.csv')
df_labeledMovies= pd.DataFrame({'movieId': df_ContBaseRec.index, 'labels': labels}).merge(df_movies, on='movieId').set_index('movieId')

# recommending movies based on several movies

In [18]:
df_movies = pd.read_csv('movies.csv')
def recommendation(given_movies, number_of_recommendations, df=df_ContBaseRec, lambda_val= np.random.rand()):
    given_movies_ids = []
    for title, year in given_movies:
        given_movie_id = df_movies[
            df_movies.title.str.contains(r'\b{}\b.*\b{}\b.$'.format(title, year), case=False)].movieId.item()
        given_movies_ids.append(given_movie_id)

    # calculating the mean point (vector) of the given movies
    mean_point = df.loc[given_movies_ids].mean()

    # movie recommendation with cousin similarity
    from sklearn.metrics.pairwise import cosine_similarity
    # Get the relevance vector of the given movie
    given_movie_vector = mean_point.values.reshape(1, -1)
    # Calculate cosine similarity between the given movie and all other movies
    similarity_scores = cosine_similarity(df.values, given_movie_vector)
    
    # distances as relevance score with nearest neighbors distances
    import sklearn.neighbors
    nn = sklearn.neighbors.NearestNeighbors(n_neighbors=df.shape[0])
    nn.fit(df)
    # getting the distances of any point in the df to mean point
    distances, indices = nn.kneighbors(mean_point.values.reshape(1, -1))
    #normalize the data between 0 and 1
    distances = (distances - distances.min()) / (distances.max() - distances.min())
   
    # Diversification
    score = pd.Series((similarity_scores.reshape(1, -1) - (lambda_val * distances))[0] , name="score")
    
    df_scored = pd.concat([df_labeledMovies.reset_index(names='movieId'), score.reset_index(drop=True)], axis=1).set_index('movieId')

    #Remove the given movie from the list of recommendation
    df_scored = df_scored.drop(given_movies_ids)
    # deleting the movies which are not in the same clusters as the given movies
    given_movies_clusters = df_labeledMovies.loc[given_movies_ids, "labels"].unique()
    df_scored = df_scored[df_scored["labels"].isin(given_movies_clusters)]
    

    #getting the movies from the labled set which shows the title and label 
    recommendations = df_scored.sort_values('score', ascending=False).head(number_of_recommendations)
    return recommendations

In [19]:
df_movies = pd.read_csv('movies.csv')

given_movies = [['Akira', '1988'], ['Network', '1976'], ['Prisoners', '2013'], ['Incendies', '2010'],
                ['red shoes', '1948'], ['metropolis', '1927']]

recommendation(given_movies, 20)

C:\Users\HamidEsfahlani\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,labels,title,genres,score
movieId,,,,
47,10,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0.803813
50,10,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0.797395
111,10,Taxi Driver (1976),Crime|Drama|Thriller,0.789949
6,10,Heat (1995),Action|Crime|Thriller,0.788315
29,4,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,0.787484
30,5,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama,0.783168
32,4,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,0.778621
36,2,Dead Man Walking (1995),Crime|Drama,0.770216
16,10,Casino (1995),Crime|Drama,0.765068


In [ ]:
def HybridRanking(X, S, lambda_val, Sim1, Sim2):
    """
    Diversifies the ranking by applying the MMR (Maximal Marginal Relevance) technique.
    
    Args:
        X (list): List of items to be ranked.
        S (list): List of already selected items.
        lambda_val (float): Parameter to adjust the trade-off between relevance and diversity.
        Sim1 (function): Function to measure the relevance between an item and the user.
        Sim2 (function): Function to measure the similarity between two items.
    
    Returns:
        list: Diversified ranking of items.
    """
    max_marginal_relevance = -float('inf')
    omega_star = None
    
    for omega_i in X:
        if omega_i not in S:
            marginal_relevance = lambda_val * Sim1(omega_i) - (1 - lambda_val) * max(Sim2(omega_i, omega_j) for omega_j in S)
            if marginal_relevance > max_marginal_relevance:
                max_marginal_relevance = marginal_relevance
                omega_star = omega_i
    
    return omega_star


# Example usage:

# Define the relevance function Sim1 and similarity function Sim2
def Sim1(item):
    # Measure the relevance between the item and the user
    pass

def Sim2(item1, item2):
    # Measure the similarity between two items
    pass

# Set the input values
X = [...]  # List of items to be ranked
S = [...]  # List of already selected items
lambda_val = ...  # Parameter to adjust the trade-off between relevance and diversity

# Apply the diversification method
diversified_ranking = diversify_ranking(X, S, lambda_val, Sim1, Sim2)